In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pathlib
import random
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
import PIL

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
images_path = os.path.join(os.path.realpath('..'), "input", "garbage-classification", "Garbage classification", "Garbage classification")
images_path = pathlib.Path(images_path)

train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

train_generator=train.flow_from_directory(images_path,
                                          target_size=(300,300),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

test_generator=test.flow_from_directory(images_path,
                                        target_size=(300,300),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='validation')

labels = (train_generator.class_indices)
print(labels)

labels = dict((v,k) for k,v in labels.items())
print(labels)

In [ ]:
num_classes = 6

model = tf.keras.Sequential([
  layers.GaussianNoise(0.01),
  layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300,300,3), padding='same'),
  layers.MaxPooling2D((2, 2), strides=(2, 2)),
    
  layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D((2, 2), strides=(2, 2)),
    
  layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D((2, 2), strides=(2, 2)),
    
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(num_classes, activation='softmax')   
])

opt = SGD(lr=0.01)
model.compile(
  optimizer="adam",
  loss=tf.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint("model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

model.fit(train_generator,
          epochs=100,
          steps_per_epoch=2276//32,
          validation_data=test_generator,
          validation_steps=251//32,
          workers = 4,
          callbacks=[checkpoint, early])

model.summary()